In [1]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams


In [2]:

t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.04096245765686035  seconds 


In [3]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [4]:
def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object

In [5]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [6]:
def read_xml(li):
  all_objects=[]
  for i in range(len(li)):
    # print("i will run ", i+1," times")
    name_of_file=clean_file_names(li[i])
    # print(name_of_file)
    data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

    post_text='  '
    tree = etree.parse(li[i], parser=parser)
    root = tree.getroot()
    for element in root.iter():
        if element.tag == 'post':
            element_text = element.text
            post_text = post_text + element_text
    mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
    mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



    data_object['posts']=mystring   
    all_objects.append(data_object)

  return all_objects



In [7]:
len(li)

14600

In [8]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [9]:
print("time taken to read :: ", (t2-t1)/100)

time taken to read ::  1.3850977206230164


In [10]:
df=pd.DataFrame(data)

In [11]:
df.tail()

,id,gender,age,category,horoscope,posts
14595,3122872,male,15,Student,Libra,ok we got webspace off abp for bgs but of cour...
14596,3322392,female,15,indUnk,Pisces,"dearest diary, i do not know wad to put for t..."
14597,4008208,female,16,indUnk,Pisces,Good friends are the greatest treasures of th...
14598,1599685,male,23,Student,Capricorn,WE BRING THIS SPECIAL ANNOUCEMENT TO YOU LIVE...
14599,1608709,male,25,Accounting,Aries,Football Alex 'Knight of the Realm' Ferguson ...


In [12]:
data=''

## Logistic regression for gender classification

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [54]:
sentences = df['posts'].values
y = df['gender'].values


In [14]:
df['gender'].replace('male',1, inplace=True)
df['gender'].replace('female',0, inplace=True)

In [15]:
y=y.astype(float)


In [55]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)


In [56]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.7791780821917809


/home/ebryx/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### END Logistic regression for gender classification

## Basic NN for gender classification

In [20]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [21]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                13249120  
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 13,249,341
Trainable params: 13,249,341
Non-trainable params: 0
_________________________________________________________________


In [24]:
classifier_nn = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Train on 10950 samples, validate on 3650 samples
Epoch 1/5
10950/10950 [==============================] - 200s 18ms/step - loss: 0.1013 - acc: 0.9721 - val_loss: 0.5401 - val_acc: 0.8107
Epoch 2/5
10950/10950 [==============================] - 190s 17ms/step - loss: 0.0947 - acc: 0.9739 - val_loss: 0.6476 - val_acc: 0.8123
Epoch 3/5
10950/10950 [==============================] - 200s 18ms/step - loss: 0.0769 - acc: 0.9779 - val_loss: 0.6583 - val_acc: 0.7945
Epoch 4/5
10950/10950 [==============================] - 193s 18ms/step - loss: 0.0679 - acc: 0.9794 - val_loss: 0.6241 - val_acc: 0.8137
Epoch 5/5
 6840/10950 [=================>............] - ETA: 1:09 - loss: 0.1122 - acc: 0.9686

KeyboardInterrupt: 

### END Basic NN for gender classification

## Multilabel classification

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [59]:
from sklearn.feature_extraction.text import CountVectorizer


In [60]:
sentences = df['posts'].values

In [61]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [62]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [63]:
labels = mlb.fit_transform(labels)

In [64]:
y=labels

In [65]:
(trainX, testX, trainY, testY) = train_test_split(sentences,y, test_size=0.2, random_state=42)

In [66]:
vectorizer = CountVectorizer()
vectorizer.fit(trainX)

X_train = vectorizer.transform(trainX)
X_test  = vectorizer.transform(testX)

In [67]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [68]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [69]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                13935920  
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 14)                14350     
Total params: 13,965,840
Trainable params: 13,

In [ ]:
classifier_nn = model.fit(X_train,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, testY),
                    batch_size=10)




Train on 11680 samples, validate on 2920 samples
Epoch 1/5





11680/11680 [==============================] - 215s 18ms/step - loss: 0.3832 - acc: 0.8571 - val_loss: 0.4109 - val_acc: 0.8571
Epoch 2/5
11680/11680 [==============================] - 217s 19ms/step - loss: 0.3818 - acc: 0.8571 - val_loss: 0.4240 - val_acc: 0.8591
Epoch 3/5
  710/11680 [>.............................] - ETA: 3:13 - loss: 0.3817 - acc: 0.8570